In [23]:
import os
import numpy as np
import pandas as pd
import sys
import re
import glob

# Before uploading the file please try to clean unwanted data from MPN, Mfg name, SPN, SN, PD

In [ ]:
# coupa addition to product master

In [ ]:
# This Program will take PO file as input and then follows the below process:
# 1) Clean the file for unwanted characters
# 2) Assign a Primary key which is of format <cdo_name>_SC<id>_<sequence of numbers>
# 3) Standardize column names for MPN, Mfg.name , PD, SPN , SN
# 4) Get MPN, Mfg.name , PD, SPN , SN the data for these columns along with Primary key
# 5) Then append above collected info to Item master list
# 6) Generate a secondary key for the Item master list (To Be Reviewed)
# 7) Drop duplicate after appending info to main Item master
# 8) Case Analysis for 32 combinations of MPN, Mfg.name , PD, SPN , SN
# 9) Map this data to original file + existing Database and populate information

In [24]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files")

In [25]:
#Upload the file from above directory
data=pd.read_csv('XXFHS_Reliant_Optum_Core_PO_Da_May19 _HN.csv')

In [26]:
data.columns

Index(['ID', 'PO Number', 'PO Line Item Number', 'DISTRIBUTOR', 'Manufacturer',
       'DISTRIBUTOR PRODUCT NUMBER', 'Manufacturer Part Number', 'Item ID',
       'Contract ID', 'Contract Type', 'Product Description',
       'UNSPSC_Segment_Code', '11 Digit NDC Code', 'Primary UOM',
       'Primary UOM Qty', 'Quantity', 'Unit Price before Discount',
       'Final_Cost ', 'Facility ID', 'Facility', 'Facility Address', 'Dept ID',
       'HOSPITAL DEPARTMENTS', 'PO Date', 'Order Type'],
      dtype='object')

In [27]:
def rm_consec_space(x):
    return re.sub(' +',' ',x)
    

In [28]:
#Clean the column names and assign the standard name
var1=data.columns.tolist()
var2=data.columns.tolist()

In [29]:
for i in range(len(var1)):
    var1[i]=var1[i].strip()
    var1[i]=rm_consec_space(var1[i])

In [30]:
for i in range(len(var1)):
    data.rename({var2[i]:var1[i]},axis=1,inplace=True)

In [31]:
col_names=data.columns.tolist()

In [32]:
col_names

['ID',
 'PO Number',
 'PO Line Item Number',
 'DISTRIBUTOR',
 'Manufacturer',
 'DISTRIBUTOR PRODUCT NUMBER',
 'Manufacturer Part Number',
 'Item ID',
 'Contract ID',
 'Contract Type',
 'Product Description',
 'UNSPSC_Segment_Code',
 '11 Digit NDC Code',
 'Primary UOM',
 'Primary UOM Qty',
 'Quantity',
 'Unit Price before Discount',
 'Final_Cost',
 'Facility ID',
 'Facility',
 'Facility Address',
 'Dept ID',
 'HOSPITAL DEPARTMENTS',
 'PO Date',
 'Order Type']

In [33]:
for col_name in col_names:
    data[col_name]=data[col_name].astype(str)
    data[col_name]=data[col_name].str.strip()
    data[col_name]=data[col_name].str.rstrip(',')
    #data[col_name]=data[col_name].str.replace(',','')
    data[col_name]=data[col_name].apply(lambda x: rm_consec_space(x))
    #data[col_name]=data[col_name].map(lambda x: re.sub(r'\W+','',x))
    data[col_name]=data[col_name].str.upper()

In [45]:
data.head()

,Primary_Key,ID,PO Number,PO Line Item Number,DISTRIBUTOR,Manufacturer,DISTRIBUTOR PRODUCT NUMBER,Manufacturer Part Number,Item ID,Contract ID,...,Quantity,Unit Price before Discount,Final_Cost,Facility ID,Facility,Facility Address,Dept ID,HOSPITAL DEPARTMENTS,PO Date,Order Type
0,Reliant_SC8-1,449733,635456,1,MCKESSON GENERAL MEDICAL CORP,MCKESSON GENERAL MEDICAL CORP,NAN,NAN,NAN,NAN,...,1.0,26.46,26.46,74.0,LEOMINSTER,"225 NEW LANCASTER ROAD, LEOMINSTER, US, 01453",40.074.2310.611210.99.9999,PEDIATRICS,1-MAY-19,STANDARD
1,Reliant_SC8-2,449733,635456,2,MCKESSON GENERAL MEDICAL CORP,MCKESSON GENERAL MEDICAL CORP,NAN,NAN,NAN,NAN,...,1.0,7.47,7.47,74.0,LEOMINSTER,"225 NEW LANCASTER ROAD, LEOMINSTER, US, 01453",40.074.2310.611210.99.9999,PEDIATRICS,1-MAY-19,STANDARD
2,Reliant_SC8-3,449733,635456,3,MCKESSON GENERAL MEDICAL CORP,MCKESSON GENERAL MEDICAL CORP,NAN,NAN,NAN,NAN,...,5.0,7.35,36.75,74.0,LEOMINSTER,"225 NEW LANCASTER ROAD, LEOMINSTER, US, 01453",40.074.2310.611210.99.9999,PEDIATRICS,1-MAY-19,STANDARD
3,Reliant_SC8-4,449733,635456,4,MCKESSON GENERAL MEDICAL CORP,MCKESSON GENERAL MEDICAL CORP,NAN,NAN,NAN,NAN,...,4.0,1.2,4.8,74.0,LEOMINSTER,"225 NEW LANCASTER ROAD, LEOMINSTER, US, 01453",40.074.2310.611210.99.9999,PEDIATRICS,1-MAY-19,STANDARD
4,Reliant_SC8-5,449733,635456,5,MCKESSON GENERAL MEDICAL CORP,MCKESSON GENERAL MEDICAL CORP,NAN,NAN,NAN,NAN,...,1.0,12.88,12.88,74.0,LEOMINSTER,"225 NEW LANCASTER ROAD, LEOMINSTER, US, 01453",40.074.2310.611210.99.9999,PEDIATRICS,1-MAY-19,STANDARD


In [35]:
cdo_name= input(prompt='Please enter CDO Name : ')

Please enter CDO Name : Reliant


In [36]:
SC_id= input(prompt='Enter the SC number: ')

Enter the SC number: 8


In [37]:
PK=cdo_name+'_SC'+SC_id+'-'

In [38]:
PK

'Reliant_SC8-'

In [39]:
data=data.reset_index()

In [41]:
data.rename({'index':'Primary_Key'},axis=1,inplace=True)

In [42]:
data['Primary_Key']=data.index+1

In [43]:
data['Primary_Key']=data['Primary_Key'].astype(str)

In [44]:
data['Primary_Key']=PK+data['Primary_Key']

# Save the file which has primary key assigned

In [46]:
data.to_csv('XXFHS_Reliant_Optum_Core_PO_Da_May19_Primary_Key_assigned.csv')

In [ ]:
data.columns

In [ ]:
data['manufacturer_part_number']=""
data['manufacturer_name']=""


In [ ]:
data['supplier_part_number']=""

# Below step values have to be manually replaced

In [ ]:
data.rename({'PARTDESC':'product_desc', 'MANPARTNUM':'manufacturer_part_number',
             'MANUFDESC':'manufacturer_name','SUPPLIERDESC':'supplier_name','SUPPARTNUM':'supplier_part_number'}
            ,axis=1,inplace=True)

In [ ]:
extract_data=data[['Primary_Key','product_desc','manufacturer_part_number','manufacturer_name','supplier_part_number','supplier_name']]

In [79]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files/product master files/All PM's with dup")

In [ ]:
pwd

In [ ]:
product_master=pd.read_csv('Product_master_stage_1_with_dup.csv')

In [ ]:
len(extract_data)

In [ ]:
product_master=product_master.append(extract_data,ignore_index=True)

In [ ]:
len(product_master)

In [ ]:
product_master.to_csv('Product_master_stage_2_with_dup.csv')

In [80]:
patch_work=pd.read_csv('Product_master_stage_2_with_dup.csv')

In [84]:
clip=pd.read_clipboard()

In [89]:
srch_list=list(clip['Primary_Key'])

In [90]:
idx_list=[]

In [91]:
for ind, row in patch_work.iterrows():
    if row['Primary_Key'] in srch_list:
        print(row['Primary_Key'])
        idx_list.append(ind)
    

DMG_TEC_SC1-8230
DMG_TEC_SC1-8315
DMG_TEC_SC1-8434
DMG_TEC_SC1-8446
DMG_TEC_SC1-8611
DMG_TEC_SC1-9471
DMG_TEC_SC1-9479
DMG_TEC_SC1-9532
DMG_TEC_SC1-9620
DMG_TEC_SC1-9725
DMG_TEC_SC1-9859
DMG_TEC_SC1-10045
DMG_TEC_SC1-10264
DMG_TEC_SC1-10492
DMG_TEC_SC1-10732
DMG_TEC_SC1-10876
DMG_TEC_SC1-11051
DMG_TEC_SC1-11728
DMG_TEC_SC1-11832
DMG_TEC_SC1-12033
DMG_TEC_SC1-12168
DMG_TEC_SC1-12201
DMG_TEC_SC1-12270
DMG_TEC_SC1-12370
DMG_TEC_SC1-12410
DMG_TEC_SC1-12434
DMG_TEC_SC1-12462
DMG_TEC_SC1-12533
DMG_TEC_SC1-12563
DMG_TEC_SC1-13150
DMG_TEC_SC1-13822
DMG_TEC_SC1-14335
DMG_TEC_SC1-18498
DMG_TEC_SC1-19238
DMG_TEC_SC1-20727
DMG_TEC_SC1-20749
DMG_TEC_SC1-22257
DMG_TEC_SC1-24413
DMG_TEC_SC1-25661
DMG_TEC_SC1-26613
DMG_TEC_SC1-26676
DMG_TEC_SC1-26682
DMG_TEC_SC1-30407
DMG_TEC_SC1-31122
DMG_TEC_SC1-33445
DMG_TEC_SC1-34279
DMG_TEC_SC1-35742


In [95]:
patch_work.drop(labels=idx_list,axis=0,inplace=True)

In [96]:
len(patch_work)

886039

In [97]:
patch_work.to_csv("Product_master_stage_3_with_dup.csv")

In [98]:
product_master=pd.read_csv('Product_master_stage_3_with_dup.csv')

In [104]:
product_master.columns

Index(['Primary_Key', 'product_desc', 'manufacturer_part_number',
       'manufacturer_name', 'supplier_part_number', 'supplier_name'],
      dtype='object')

In [103]:
product_master.drop(labels=['S. No'],axis=1,inplace=True)

In [106]:
product_master.replace(to_replace=np.nan,value='NAN',inplace=True)

In [99]:
len(product_master)

886086

In [ ]:
pwd

In [107]:
f = {c: ','.join if c == 'Primary_Key' else 'first' for c in ['Primary_Key']}

product_master=product_master.groupby(['product_desc', 'manufacturer_part_number','manufacturer_name', 'supplier_part_number', 'supplier_name']).agg(f).reset_index()

In [108]:
len(product_master)

178094

In [109]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files/product master files/All PM's without dup")

In [110]:
product_master.to_csv('product_master_stage_3_without_duplicates.csv')

# Clean the without duplicate file

In [2]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Ganesh_Merged_Files/Merged files duplicates/Unique item master files")

In [3]:
to_clean=pd.read_csv("All_files_merged_no_dup_with_spend_new_2.csv")

c:\users\ganesh p umarani\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(to_clean)

325075

In [6]:
to_clean.replace(to_replace=np.nan,value='NAN',inplace=True)

In [11]:
to_clean.dtypes

Secodary_key        int64
Dist_part_num_x    object
Distributor_x      object
Manufacturer_x     object
Mfg_part_num_x     object
Prod_desc_x        object
dtype: object

In [10]:
to_clean.drop(labels=['Final_cost_x'],axis=1,inplace=True)

In [12]:
grouped_res= to_clean.groupby(['Dist_part_num_x','Mfg_part_num_x'])

In [15]:
df_grpby_res=grouped_res.count()

In [16]:
df_grpby_res=pd.DataFrame(df_grpby_res.to_records())

In [22]:
len(df_grpby_res)

14354

In [19]:
list_cols

['Dist_part_num_x',
 'Mfg_part_num_x',
 'Secodary_key',
 'Distributor_x',
 'Manufacturer_x',
 'Prod_desc_x']

In [18]:
list_cols=list(df_grpby_res.columns)

In [83]:
# df_grpby_res.drop(labels=list_cols,axis=1,inplace=True)

In [21]:
df_grpby_res=df_grpby_res[df_grpby_res['Prod_desc_x']>1]

In [30]:
fin_res_smae_mpn_spn_combo=pd.DataFrame(columns=['Secondary_key','manufacturer_part_number','supplier_part_number','product_desc','manufacturer_name','supplier_name'])

In [112]:
a=df_grpby_res['manufacturer_part_number'][0]
b=df_grpby_res['supplier_part_number'][0]
recs=to_clean[(to_clean['manufacturer_part_number']==a)&(to_clean['supplier_part_number']==b)]

In [120]:
recs

,product_desc,manufacturer_part_number,manufacturer_name,supplier_part_number,supplier_name,Primary_Key
62609,EMBROIDERY DEPTARTMENT NAME,0,MEDLINE,0,000059 / MEDLINE INDUSTRIES INC,"Mountain_west_SC1-18621,Mountain_west_SC1-1862..."
62616,EMBROIDERY PROVIDER NAME,0,MEDLINE,0,000059 / MEDLINE INDUSTRIES INC,"Mountain_west_SC1-12668,Mountain_west_SC1-1266..."


In [26]:
to_clean.columns

Index(['Secodary_key', 'Dist_part_num_x', 'Distributor_x', 'Manufacturer_x',
       'Mfg_part_num_x', 'Prod_desc_x'],
      dtype='object')

In [31]:
fin_res_smae_mpn_spn_combo.columns

Index(['Secondary_key', 'manufacturer_part_number', 'supplier_part_number',
       'product_desc', 'manufacturer_name', 'supplier_name'],
      dtype='object')

In [37]:
df_grpby_res.drop(labels=['index'],axis=1,inplace=True)

c:\users\ganesh p umarani\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
df_grpby_res.columns

Index(['Dist_part_num_x', 'Mfg_part_num_x', 'Secodary_key', 'Distributor_x',
       'Manufacturer_x', 'Prod_desc_x'],
      dtype='object')

In [40]:
df_grpby_res['Mfg_part_num_x'][0]

'NAN'

In [41]:
for i in range(len(df_grpby_res)):
    a=df_grpby_res['Mfg_part_num_x'][i]
    b=df_grpby_res['Dist_part_num_x'][i]
    records=to_clean[(to_clean['Mfg_part_num_x']==a)&(to_clean['Dist_part_num_x']==b)]
    for index, row in records.iterrows():
        fin_res_smae_mpn_spn_combo=fin_res_smae_mpn_spn_combo.append({'manufacturer_part_number':row['Mfg_part_num_x'],
                                                                 'supplier_part_number':row['Dist_part_num_x'],
                                                                 'product_desc':row['Prod_desc_x'],
                                                                 'supplier_name':row['Distributor_x'],
                                                                 'manufacturer_name':row['Manufacturer_x'],
                                                                 'Secondary_key':row['Secodary_key']},ignore_index=True)
    

In [42]:
fin_res_smae_mpn_spn_combo.to_csv('fin_res_smae_mpn_spn_combo_1.csv')

In [60]:
df_grpby_res.to_csv('same_mpn_spns.csv')

In [68]:
df_grpby_res=df_grpby_res[df_grpby_res['manufacturer_part_number']==df_grpby_res['supplier_part_number']]

# Case Analysis

In [ ]:
from itertools import combinations

In [ ]:
count_list=[]

In [ ]:
product_master.replace(to_replace='NAN',value=np.nan,inplace=True)
product_master.replace(to_replace='',value=np.nan,inplace=True)

count_product_desc=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc))

count_manufacturer_part_number=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_part_number))

count_manufacturer_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_name))

count_supplier_part_number=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_supplier_part_number))

count_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_supplier_name))

count_product_desc_manufacturer_part_number=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_part_number))

count_product_desc_manufacturer_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_name))

count_product_desc_supplier_part_number=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_supplier_part_number))

count_product_desc_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_supplier_name))

count_manufacturer_part_number_manufacturer_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_part_number_manufacturer_name))

count_manufacturer_part_number_supplier_part_number=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_part_number_supplier_part_number))

count_manufacturer_part_number_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_part_number_supplier_name))

count_manufacturer_name_supplier_part_number=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_name_supplier_part_number))

count_manufacturer_name_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_name_supplier_name))

count_supplier_part_number_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_supplier_part_number_supplier_name))

count_product_desc_manufacturer_part_number_manufacturer_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_manufacturer_name))

count_product_desc_manufacturer_part_number_supplier_part_number=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_supplier_part_number))

count_product_desc_manufacturer_part_number_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_supplier_name))

count_product_desc_manufacturer_name_supplier_part_number=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_name_supplier_part_number))

count_product_desc_manufacturer_name_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_name_supplier_name))

count_product_desc_supplier_part_number_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_supplier_part_number_supplier_name))

count_manufacturer_part_number_manufacturer_name_supplier_part_number=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_manufacturer_part_number_manufacturer_name_supplier_part_number))

count_manufacturer_part_number_manufacturer_name_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_part_number_manufacturer_name_supplier_name))

count_manufacturer_part_number_supplier_part_number_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_part_number_supplier_part_number_supplier_name))

count_manufacturer_name_supplier_part_number_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_name_supplier_part_number_supplier_name))

count_product_desc_manufacturer_part_number_manufacturer_name_supplier_part_number=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.isnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_manufacturer_name_supplier_part_number))

count_product_desc_manufacturer_part_number_manufacturer_name_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.isnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_manufacturer_name_supplier_name))

count_product_desc_manufacturer_part_number_supplier_part_number_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.isnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_supplier_part_number_supplier_name))

count_product_desc_manufacturer_name_supplier_part_number_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.isnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_name_supplier_part_number_supplier_name))

count_manufacturer_part_number_manufacturer_name_supplier_part_number_supplier_name=product_master[(product_master.product_desc.isnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_manufacturer_part_number_manufacturer_name_supplier_part_number_supplier_name))

count_product_desc_manufacturer_part_number_manufacturer_name_supplier_part_number_supplier_name=product_master[(product_master.product_desc.notnull())&(product_master.manufacturer_part_number.notnull())&(product_master.manufacturer_name.notnull())&(product_master.supplier_part_number.notnull())&(product_master.supplier_name.notnull())]
count_list.append(len(count_product_desc_manufacturer_part_number_manufacturer_name_supplier_part_number_supplier_name))

In [ ]:
matrix=pd.DataFrame(columns=['Combination','Count of Cases'])
for i in range(1,6,1):
    comb = list(combinations(['product_desc', 'manufacturer_part_number', 'manufacturer_name','supplier_part_number', 'supplier_name'],i))
    for j in range(len(comb)):
        matrix=matrix.append({'Combination': comb[j]}, ignore_index=True)
        

In [ ]:
for i in range(len(count_list)):
    matrix['Count of Cases'][i]=count_list[i]

In [ ]:
matrix

In [ ]:
matrix.to_excel('Case_Analysis_14_PO_files_3.xlsx')

#Manufacturer based break-up of cases

In [ ]:
product_master_pd_mpn_mfg_name=prod_master_with_duplicates.groupby(['product_desc', 'manufacturer_part_number','manufacturer_name']).agg(f).reset_index()

In [ ]:
len(product_master_pd_mpn_mfg_name)

In [ ]:
count_list_by_mfg

In [ ]:
product_master_pd_mpn_mfg_name.replace(to_replace='',value=np.nan,inplace=True)

In [ ]:
product_master_pd_mpn_mfg_name.replace(to_replace='NAN',value=np.nan,inplace=True)

In [ ]:
len(product_master_pd_mpn_mfg_name)

In [ ]:
product_master_pd_mpn_mfg_name.to_csv('without_duplicate_by_mfg.csv')

In [ ]:
count_list_by_mfg=[]

In [ ]:
cnt_pd=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.isnull())]
count_list_by_mfg.append(len(cnt_pd))

In [ ]:
cnt_mfg_name=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.isnull())]
count_list_by_mfg.append(len(cnt_mfg_name))

cnt_mpn=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.notnull())]
count_list_by_mfg.append(len(cnt_mpn))

cnt_pd_mfg_name=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.isnull())]
count_list_by_mfg.append(len(cnt_pd_mfg_name))

cnt_pd_mpn=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.notnull())]
count_list_by_mfg.append(len(cnt_pd_mpn))

cnt_mfg_name_mpn=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.notnull())]
count_list_by_mfg.append(len(cnt_mfg_name_mpn))

cnt_pd_mfg_name_mpn=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.notnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.notnull())]
count_list_by_mfg.append(len(cnt_pd_mfg_name_mpn))

cnt_none_of_the_three=product_master_pd_mpn_mfg_name[(product_master_pd_mpn_mfg_name.product_desc.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_name.isnull())&(product_master_pd_mpn_mfg_name.manufacturer_part_number.isnull())]
count_list_by_mfg.append(len(cnt_none_of_the_three))

In [ ]:
matrix_by_mfg=pd.DataFrame(columns=['Combination','Count of Cases'])
for i in range(1,4,1):
    comb = list(combinations(['product_desc', 'manufacturer_name', 'manufacturer_part_number'],i))
    for j in range(len(comb)):
        matrix_by_mfg=matrix_by_mfg.append({'Combination': comb[j]}, ignore_index=True)

In [ ]:
for i in range(len(count_list_by_mfg)):
    matrix_by_mfg['Count of Cases'][i]=count_list_by_mfg[i]

In [ ]:
matrix_by_mfg['Count of Cases'].sum()

In [ ]:
matrix_by_mfg.to_csv('case_analysis_by_mfg_2.csv')

# Now after dropping the duplicates from the product master and getting the case analysis
# We will index each row from all the PO file into Elastic search

In [62]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files/PO files with primary key")

In [39]:
glob.glob("*.*")

['Copy of UHG_EXP_REPORT_W _DELIVERY_INFO_114511_155343_Primary_Key_assigned.csv',
 'COUPA_PO_line_list-cube - Feb 2019-May 2019_Primary_Key_assigned.csv',
 'FHSC Purchase History_5mo_Jan 2019-May 2019_Primary_Key_assigned.csv',
 'Optum Request 1 (POs) (1)_Primary_Key_assigned.csv',
 'OptumCare-PO Data Jan18_May19_Primary_Key_assigned.csv',
 'PO Data_DMG Markets_1-1-18 to 6-30-19_Updated GPO+Non GPO_Primary_Key_assigned.csv',
 'Reliant_PO_06012019_06302019_June19_Primary_Key_assigned.csv',
 'Reliant_PO_07012019_07312019_V1_Primary_Key_assigned.csv',
 'SCA_Purchase_Pull_CSV_Primary_Key_assigned.csv',
 'TEC_PO_01012018_07312019_Primary_Key_assigned.csv',
 'TPC Purchase History_5mo_Jan 2019-June 2019_Primary_Key_assigned.csv',
 'TPC_PO_01012018_12312018_v1_Primary_Key_assigned.csv',
 'USMD File Files-Cube Request PO_Tied_To_Vendor_Bill_Line Item  Jan-2019 -v2_Primary_Key_assigned.csv',
 'XXFHS_Reliant_Optum_Core_PO_Da_Jan18_Dec18_Primary_Key_assigned.csv']

In [63]:
po_file=pd.read_csv('SCA_Purchase_Pull_CSV_Primary_Key_assigned.csv')

In [64]:
len(po_file)

65499

In [69]:
po_file.columns

Index(['Primary_Key', 'ID', 'Category', 'FACILITY', '6DIGIT', 'PO Number',
       'Invoice number', 'DISTRIBUTOR', 'Manufacturer', 'FINALNAME',
       'DISTRIBUTOR PRODUCT NUMBER', 'ALPHASUPPARTNUM',
       'Manufacturer Part Number', 'ALPHAMANPARTNUM', 'FINALPARTNUM',
       'Product Description', 'QOE', 'Quantity', 'Primary UOM',
       'Unit Price after Discount', ' Final_Cost ', 'GLACCT', 'PO DATE',
       'ORDERTYPE', 'AHFS_NUMBER', 'AHFS_DESCRIPTION', 'NDC', 'ITEMID',
       'CLEANITEMID', 'CLEANCATMASTERID', 'CLEANDATE', 'BUYRIGHTNAME',
       'BUYRIGHTSTATUS', 'SEQ', 'RECEIVEDDATE'],
      dtype='object')

In [66]:
po_file.drop(labels='Unnamed: 0',axis=1,inplace=True)

In [68]:
po_file.rename({'PO#':'PO Number','INVOICENUM':'Invoice number','SUPPLIERDESC':'DISTRIBUTOR','MANUFDESC':'Manufacturer',
                'SUPPARTNUM':'DISTRIBUTOR PRODUCT NUMBER','MANPARTNUM':'Manufacturer Part Number',
               'PARTDESC':'Product Description', 'PACKQTY':'QOE', 'QTY':'Quantity', 'UOM':'Primary UOM',
               'QUOTEDPRICE':'Unit Price after Discount', 'TOTAL':' Final_Cost '},axis=1,inplace=True)

In [70]:
po_file.head()

,Primary_Key,ID,Category,FACILITY,6DIGIT,PO Number,Invoice number,DISTRIBUTOR,Manufacturer,FINALNAME,...,AHFS_DESCRIPTION,NDC,ITEMID,CLEANITEMID,CLEANCATMASTERID,CLEANDATE,BUYRIGHTNAME,BUYRIGHTSTATUS,SEQ,RECEIVEDDATE
0,SCA_SC1-1,31862697,42132205.0,50173,50173,H2020546,1871593185,MEDLINE,MEDLINE,MEDLINE,...,NAN,NAN,1527763,NAN,NAN,NAN,SURGICAL GLOVES,Y,29,2019-03-06 00:25:05.000;;;;;;
1,SCA_SC1-2,31862947,42311512.0,50441,50441,H2020704,1871590718,MEDLINE,MEDLINE,MEDLINE,...,NAN,NAN,1580765,NAN,NAN,NAN,NAN,Y,7,2019-03-06 00:20:05.000;;;;;;;
2,SCA_SC1-3,31863029,42131606.0,50104,50104,H2020611,1871627581,MEDLINE,MEDLINE,MEDLINE,...,NAN,NAN,197157,NAN,NAN,NAN,SURGICAL MASKS,Y,19,2019-03-06 01:05:06.000;;;;;;;
3,SCA_SC1-4,31863301,42312201.0,50373,50373,H2020667,A1947380,SUTURE EXPRESS,ETHICON,ETHICON,...,NAN,NAN,70108,NAN,NAN,NAN,SURGICAL SUTURES,Y,15,2019-03-05 08:10:06.000;;;;;;
4,SCA_SC1-5,31863405,42131702.0,50373,50373,H2020663,1871639316,MEDLINE,MEDLINE,MEDLINE,...,NAN,NAN,241243,NAN,NAN,NAN,DRAPES & GOWNS,Y,27,2019-03-06 01:20:06.000;;;;;;;;


In [71]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files/PO Files used in ES")

In [72]:
#Export this file to above folder for future use/reference
po_file.to_csv('SCA_Purchase_Pull_CSV_Primary_Key_assigned_ES.csv')

In [73]:
# 1) get each PO file with standardized column headers
# 2) conver data type of each column to String
# 3) Index it into ES

In [111]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Optum_Raw_Initial_Conversion/PO Files/PO Files used in ES")

In [112]:
files_for_ES= glob.glob("*.*")

In [298]:
file=files_for_ES[14]

IndexError: list index out of range

In [287]:
file

'XXFHS_Reliant_Optum_Core_PO_Da_Jan18_Dec18_Primary_Key_assigned_ES.csv'

In [288]:
data=pd.read_csv(file)
col_names=data.columns.tolist()
for col_name in col_names:
    data[col_name]=data[col_name].astype(str)
    data[col_name]=data[col_name].str.strip()
    data[col_name]=data[col_name].str.rstrip(',')
    #data[col_name]=data[col_name].str.replace(',','')
    data[col_name]=data[col_name].apply(lambda x: rm_consec_space(x))
    #data[col_name]=data[col_name].map(lambda x: re.sub(r'\W+','',x))
    data[col_name]=data[col_name].str.upper()
    

In [289]:
data.columns

Index(['Unnamed: 0', 'Primary_Key', 'ID', 'PO Number', 'PO Line Item Number',
       'DISTRIBUTOR', 'Manufacturer', 'DISTRIBUTOR PRODUCT NUMBER',
       'Manufacturer Part Number', 'Item ID', 'CONTRACT NUMBER',
       'Contract Type', 'Product Description', 'UNSPSC_Segment_Code',
       '11 Digit NDC Code', 'Primary UOM', 'Primary UOM Qty', 'Quantity',
       'Unit Price before Discount', 'Final_Cost', 'Facility', 'Facility Name',
       'Facility Address', 'Dept ID', 'HOSPITAL DEPARTMENTS', 'PO Date',
       'Order Type'],
      dtype='object')

In [290]:
data.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [291]:
col_names.pop(0)

'Unnamed: 0'

In [292]:
for col_name in col_names:
    print(type(data[col_name][0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [2]:
es = Elasticsearch()

In [3]:
es.indices.get_alias("*")

{'po_file': {'aliases': {}},
 '.kibana_task_manager': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}}}

In [293]:
es.indices.refresh('po_file')
es.cat.count('po_file')

'1569508709 14:38:29 855838\n'

In [294]:
len(data)

40860

In [295]:
documents = data.to_dict(orient='records')
bulk(es, documents, index='po_file',doc_type='raw_files', raise_on_error=True)

(40860, [])

In [296]:
res = es.search(index="po_file",body = {
    "query": {
        "match_phrase" : {
            "Primary_Key" : 'Reliant_SC3-1'
        }
    }
})

In [297]:
res

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 14.120883,
  'hits': [{'_index': 'po_file',
    '_type': 'raw_files',
    '_id': 'C14Cbm0BvoFd6VeWeEB9',
    '_score': 14.120883,
    '_source': {'Primary_Key': 'RELIANT_SC3-1',
     'ID': '380639',
     'PO Number': '623899',
     'PO Line Item Number': '1',
     'DISTRIBUTOR': 'MCKESSON GENERAL MEDICAL CORP',
     'Manufacturer': 'MCKESSON GENERAL MEDICAL CORP',
     'DISTRIBUTOR PRODUCT NUMBER': 'NAN',
     'Manufacturer Part Number': '1232',
     'Item ID': '12196',
     'CONTRACT NUMBER': 'NAN',
     'Contract Type': 'NAN',
     'Product Description': '234181 BOWL, 1 QUART STERILE DISPOSABLE',
     'UNSPSC_Segment_Code': 'NAN',
     '11 Digit NDC Code': 'NAN',
     'Primary UOM': 'CASE',
     'Primary UOM Qty': '5.0',
     'Quantity': '5.0',
     'Unit Price before Discount': '36.4',
     'Final_Cost': '182.0',
   